<a href="https://www.kaggle.com/code/badr2211/x-ray-img-classification?scriptVersionId=151886430" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2
import os
import pandas as pd
import glob
from sklearn.metrics import f1_score


In [ ]:
def get_data(p):
    label=[]
    filepath=[]
    imgs=[]
    sdir= p
    slist=os.listdir(sdir)
    for klass in slist[:2]:
        class_path=os.path.join(sdir, klass)
        #flist=os.listdir(class_path)
        flist = glob.glob(class_path+'/*')
        for f in flist:
            try :
                #fpath=os.path.join(class_path,f)
                filepath.append(f)

                img_arr = cv2.imread(f,0)#plt.imread(f)
                img_arr = cv2.resize(img_arr, (150, 150)) # Reshaping images to preferred size
                img_arr = np.array(img_arr)/255
                img_arr = np.expand_dims(img_arr, axis=-1)  # Add a channel dimension
                label.append((1 if klass =='PNEUMONIA' else 0))
                imgs.append(img_arr)
            except Exception as e:
                print(e)
    Fseries=pd.Series(filepath, name='filepaths')
    Lseries=pd.Series(label, name='labels')
    df=pd.concat([Fseries, Lseries], axis=1)

    print(df['labels'].value_counts())

    return df ,imgs

In [ ]:
train_df,train_img =get_data('../input/chest-xray-pneumonia/chest_xray/chest_xray/train')
test_df,test_img = get_data('../input/chest-xray-pneumonia/chest_xray/chest_xray/test')
val_df,val_img = get_data('../input/chest-xray-pneumonia/chest_xray/chest_xray/val')

In [ ]:
def displot (df):
    ser=df['labels']
    
    print(ser.value_counts())
    ser.value_counts().plot.bar(color =['b','g'])   
    
    plt.show()

In [ ]:
displot(train_df)

In [ ]:
train_df#.shape,len(train_img)
#train_img


In [ ]:
plt.figure(figsize=(20,8))
for i in range(1,11):
    plt.subplot(2,5,i)
    plt.imshow(train_img[i*400],'gray')
    plt.title(train_df.iloc[i*100,1])
        
plt.show()

In [ ]:

x_train = np.array(train_img)
x_val = np.array(val_img)
x_test = np.array(test_img)

y_train = np.array(train_df.iloc[:,1])
y_val = np.array(val_df.iloc[:,1])
y_test = np.array(test_df.iloc[:,1])


### We use ```imagedataGenerator``` to increase our data

In [ ]:

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

### we use ```class_weight ```to deal with imbalnced classes

In [ ]:
from sklearn.utils import compute_class_weight


from sklearn.utils import class_weight
import numpy as np

class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_gen.classes),
                                        y = train_gen.classes                                                    
                                    )
class_weights = dict(zip(np.unique(train_gen.classes), class_weights))
class_weights

In [ ]:

from collections import Counter
counter = Counter(train_gen.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}

In [ ]:
train_df.iloc[:,1].value_counts()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model_1 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Assuming binary classification
])

model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history = model_1.fit(datagen.flow(x_train,y_train ,batch_size = 32)
                      ,epochs=20,
                      validation_data =( x_val, y_val),class_weight=class_weights)

In [ ]:
probabilities = model_1.predict(x_test)#( test_gen)

# Create binary predictions (1 or 0) based on a threshold (e.g., 0.5)
y_pred = [1 if prob > 0.5  else 0 for prob in probabilities]
pd.Series(y_pred ).value_counts()



In [ ]:
print("Loss of the model is - " , model_1.evaluate (x_test,y_test))

In [ ]:
print(classification_report(y_test, y_pred, target_names = ['Pneumonia (Class 0)','Normal (Class 1)']))

#### another attemp 
another model is n't do very well

In [ ]:
model = Sequential()
model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (150,150,1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 128 , activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 1 , activation = 'sigmoid'))
model.compile(optimizer = "rmsprop" , loss = 'binary_crossentropy' , metrics = ['accuracy'])
model.summary()


In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)

In [ ]:
history = model.fit(datagen.flow(x_train,
                                 #train_img,
                                 y_train ,
                                 
                                 batch_size = 32) ,
                                 class_weight=class_weights,
                                 verbose=1,
                                 epochs = 12, 
                                 validation_data = datagen.flow(x_val, y_val),
                                 callbacks = [learning_rate_reduction]
                              #  callbacks = [learning_rate_reduction]
                   )

In [ ]:
# Assuming model.predict(x_test) returns a list of probabilities
probabilities = model.predict(x_test)#( test_gen)

# Create binary predictions (1 or 0) based on a threshold (e.g., 0.5)
y_pred = [1 if prob > 0.4  else 0 for prob in probabilities]
pd.Series(y_pred ).value_counts()


In [ ]:
pd.Series(y_test).value_counts()
#y_pred

In [ ]:
print("Loss of the model is - " , model.evaluate (x_test,y_test))

In [ ]:
print(classification_report(y_test, y_pred, targeat_names = ['Pneumonia (Class 0)','Normal (Class 1)']))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:

#from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

# Create a heatmap for the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Class 0", "Class 1"],
            yticklabels=["Class 0", "Class 1"])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
